I'm a self-hosted person, I own a VPS and have installed some applications that I use in my day-to-day activities.
Those apps (like chat server, using [Matrix](https://matrix.org/)) are becoming a part of my life, and losing the data that's been there will be disastrous for me.
Hence, backup is a must thing to do.

Doing backup isn't that hard, especially if we use the right tools.
For the past few years, I've been using [borg](https://www.borgbackup.org/) as a tool for doing backup of all my computer-related data. It's a simple and easy-to-use tool. And combine with [borgmatic](https://torsion.org/borgmatic/), a program on top of borg to make the backup process even easier, backing up a whole data, both from a database and from the file, is as simple as running one command. Borgmatic use a configuration file to do the whole process. Here is my simple configuration that I used on regular basis.

**Folder Structure**

Before moving on to the configuration, I'd like to share the folder structure of my system.
I normally have 2 folders in the home folder, the `backup` folder and the `borg` folder.

```
-/home/me
--- backup/
--- borg/
```

The `backup` folder is a regular folder, that contains all the things that I want to backup.
For example, my `nginx` configuration is in here (the whole `/etc/nginx` folder), and then I link it to the `/etc`. The same with my wireguard configuration folder. 
This is all my configurations, scripts, data, and anything that I want to backup.

The second folder, `borg`, is a borg repository. Borg works by creating a repository and adding data to it. This is where your backup data will be, but not in the same format. You can't directly look into this folder. Borg doing some magical things to compress your data and to remember your past data, so it could only update whenever new/updated data is being saved. To create a borg repo with a password, you could use the command:
`borg init --encryption=repokey ~/borg/`.

**Configuration**

In the `backup` folder, apart from the things that I want to back up, I also put my borgmatic configuration in here.
In most cases, my borgmatic configuration is pretty simple, with only 5-6 things that need to be set up.
Borgmatic itself uses YAML data format for its configuration, so it's easy to understand.

```yaml
location:
    source_directories:
        - /home/me/backup
    repositories:
        - /home/me/borg

storage:
    encryption_passphrase: "mYl1Tl3SecR3T"
    borg_base_directory: /home/me/
    archive_name_format: 'auto-{now:%Y-%m-%d}'

retention:
    keep_weekly: 2
    keep_monthly: 3
    prefix: auto

consistency:
    prefix: auto

hooks:
    before_backup:
        - echo "Starting a backup."
    after_backup:
        - echo "Finished a backup."
    on_error:
        - echo "Error during prune/compact/create/check."
    after_everything:
        - rclone sync --progress /home/me/borg/ S3:/backup/borg/
```


The first thing that you need to specify is the location of the folder we want to backup and the repository.
With my structure, this means the `~/backup` and `~/borg` folder.

The second one is the repository configuration, like the password we are using in the repository and the name of the archive. Here, for the archive name, I'm using the prefix `auto-`.

Next is retention, how many archives we would like to keep. For example, in the configuration above, I want to have 2 weekly archive, and 3 monthly archive. So if for the new week, I already have 2 weeks of archive, the borgmatic will automatically prune/delete the oldest one.

After setting the prefix for consistency check, the last one is to set up a hook. A hook is a command that will be run in several stages. Here, I create a command to be run before backup, after backup, on an error, and after everything.
For the sample above, the command is quite simple, just echoing something. But in some cases, you could add a more complex thing. For example, in before_backup, I normally have a command to dump my Postgres database into the `backup` folder, so it could be backed up as well.

And also, the after everything hooks, I set up a [rclone](https://rclone.org/) process to copy the `borg` folder to somewhere else, like in an S3. 
This will help me in case of hard disk failure, where I could not retrieve the local `borg` repository, then I still have the data on the cloud.

Finally, after the configuration is done, you could do the backup by running: `borgmatic -c borgmatic_config.yaml`.